In [1]:
!pip install flask flask-cors pyngrok google-generativeai

In [2]:
import google.generativeai as genai
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok

# 🔑 Replace with your real Gemini API key from https://makersuite.google.com/app/apikey
GEMINI_API_KEY = "Your_Gemini_API_Key"
genai.configure(api_key=GEMINI_API_KEY)

/usr/local/lib/python3.12/dist-packages/google/colab/_import_hooks/_hook_injector.py:55: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  loader.exec_module(module)


In [3]:
MODEL_NAME = "models/gemini-2.5-flash"  # or models/gemini-1.0-pro if needed

In [4]:
def safe_gemini_call(prompt):
    print("🧠 USING MODEL:", MODEL_NAME)
    try:
        model = genai.GenerativeModel(MODEL_NAME)
        response = model.generate_content(prompt)
        return response.text.strip()
    except Exception as e:
        return f"AI_ERROR: {str(e)}"

In [5]:
# AGENT 1: USER CLASSIFICATION
# ===============================

def classify_user(age: int):
    if age <= 15:
        return "School Student (≤10th)"
    elif 16 <= age <= 18:
        return "Senior School (11th–12th)"
    else:
        return "College Student"


# ===============================
# AGENT 2: PERSONALITY & INTEREST ANALYSIS
# ===============================

def analyze_personality(answers: str):
    prompt = f"""
    The following are a student's responses about their interests and thinking style:
    {answers}

    Identify the top 3 personality or cognitive traits
    (e.g., analytical, creative, social, logical, empathetic).
    Describe them briefly in 2-3 lines.
    """
    return safe_gemini_call(prompt)


# ===============================
# AGENT 3: CAREER RECOMMENDATION
# ===============================

def recommend_career(traits: str, answers: str):
    prompt = f"""
    Based on these traits:
    {traits}

    And these interests or preferences:
    {answers}

    Suggest ONE career path that best fits the student.

    Respond STRICTLY in JSON with keys:
    - career_name
    - reasoning

    Example:
    {{ "career_name": "Data Scientist", "reasoning": "..." }}
    """
    return safe_gemini_call(prompt)


# ===============================
# AGENT 4: DYNAMIC SKILL GAP ANALYSIS
# ===============================

def analyze_skill_gap_dynamic(career_json: str, user_skills: list):
    prompt = f"""
    The following is a career recommendation:
    {career_json}

    List 6-8 core technical or soft skills required
    to succeed in this career.

    Respond as a comma-separated list.
    """

    skills_text = safe_gemini_call(prompt)

    required_skills = [
        s.strip().title()
        for s in skills_text.split(",")
        if s.strip()
    ]

    user_skills_norm = [s.strip().title() for s in user_skills]

    missing_skills = [
        skill for skill in required_skills
        if skill not in user_skills_norm
    ]

    return {
        "required_skills": required_skills,
        "missing_skills": missing_skills
    }


# ===============================
# AGENT 5: COURSE RECOMMENDATION
# ===============================

def suggest_courses_dynamic(missing_skills: list):
    if not missing_skills:
        return (
            "You already have all major skills required for your ideal career. "
            "Focus on projects, internships, and real-world practice."
        )

    prompt = f"""
    Suggest one beginner-friendly online course for EACH of these skills:
    {', '.join(missing_skills)}.

    For every skill, include:
    - Course title
    - Platform name (Coursera, Udemy, edX, etc.)
    """

    return safe_gemini_call(prompt)

In [6]:
def conversation_controller(age, messages):
    """
    Decides the next question based on:
    - user age
    - education category
    - conversation so far
    """

    category = classify_user(age)

    # Convert messages into readable text for Gemini
    convo_text = "\n".join(
        [f"{m['role']}: {m['content']}" for m in messages]
    )

    # ---------- SCHOOL STUDENTS ----------
    if category.startswith("School"):
        prompt = f"""
        You are a friendly career counsellor for SCHOOL students.

        Conversation so far:
        {convo_text}

        Ask ONE thoughtful question to understand:
        - interests
        - curiosity
        - thinking style

        DO NOT ask about skills or jobs.
        Ask only ONE question.
        """
        return safe_gemini_call(prompt)

    # ---------- 11th–12th STUDENTS ----------
    if category.startswith("Senior"):
        prompt = f"""
        You are a career counsellor for 11th–12th students.

        Conversation so far:
        {convo_text}

        Ask ONE question to understand:
        - preferred subjects
        - strengths (math, biology, arts, commerce, etc.)
        - career inclination

        DO NOT ask about professional skills.
        Ask only ONE question.
        """
        return safe_gemini_call(prompt)

    # ---------- COLLEGE STUDENTS ----------
    if category == "College Student":
        prompt = f"""
        You are a career counsellor for COLLEGE students.

        Conversation so far:
        {convo_text}

        If the student has NOT mentioned skills yet,
        ask about their current skills.

        Otherwise, ask one clarifying question about
        their career goals or interests.

        Ask only ONE question.
        """
        return safe_gemini_call(prompt)

In [8]:
# ===============================
# CAREERLENS CHATBOT (CLI)
# Uses your existing agents
# ===============================

print("\n🎓 CareerLens AI – Career Counsellor\n")

age = int(input("Enter your age: "))
category = classify_user(age)

print("\nAI: Great! Let's talk about you.\n")

messages = []
answers = []

for i in range(5):   # ask 5 questions
    reply = conversation_controller(age, messages)

    print("AI:", reply)
    user = input("You: ")

    messages.append({"role": "assistant", "content": reply})
    messages.append({"role": "user", "content": user})
    answers.append(user)

# Combine all user answers
final_answers = " ".join(answers)

print("\n🔍 ANALYZING YOU...\n")

traits = analyze_personality(final_answers)
career = recommend_career(traits, final_answers)

print("🧠 Personality Traits:\n", traits)
print("\n🎯 Best Career:\n", career)

# Only for college students
if category == "College Student":
    skills = input("\nEnter your current skills (comma separated): ")
    skill_list = [s.strip() for s in skills.split(",")]

    gap = analyze_skill_gap_dynamic(career, skill_list)
    courses = suggest_courses_dynamic(gap["missing_skills"])

    print("\n📌 Required Skills:", gap["required_skills"])
    print("❌ Missing Skills:", gap["missing_skills"])
    print("\n📚 Courses to Learn:\n", courses)

print("\n🎉 CareerLens Analysis Complete!")


🎓 CareerLens AI – Career Counsellor

Enter your age: 22

AI: Great! Let's talk about you.

🧠 USING MODEL: models/gemini-2.5-flash
AI: Hello! To help me understand what might be a good fit for you, could you tell me a bit about any skills you've developed so far, whether from classes, hobbies, or part-time jobs?
You: i am presently pursuing M.Tech in AI&DS and have done B.Tech in AI&DS so i have some knowledge about agentic AI . i have made some data analysis projects as well. i also made a deep learning project which was a image segmentation model and i also have some knowledge about data structures and algorithms.
🧠 USING MODEL: models/gemini-2.5-flash
AI: Excellent! With your experience in both data analysis and deep learning, are you leaning more towards roles focused on building predictive models, or perhaps more on developing intelligent systems and agents?
You: in my b.tech days i was more inclined towards machine learning and data analysis projects but recently i started learni

ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (::1) 3152.01ms


AI: That's a fantastic blend of interests! To clarify, do you envision your ideal career path leaning more towards pushing the theoretical boundaries of Agentic AI through research, or primarily focusing on engineering and deploying these agents for practical applications?
You: i want to develop agentic ai projects which can be used in real life and are based on real life applications but these application areas should be the ones which have been not researched much
🧠 USING MODEL: models/gemini-2.5-flash
AI: That's a very specific and exciting niche! When you think about 'under-researched application areas' for Agentic AI, are there any particular industries or types of problems that come to mind?
You: i prefer to work more in education and healthcare areas

🔍 ANALYZING YOU...

🧠 USING MODEL: models/gemini-2.5-flash
🧠 USING MODEL: models/gemini-2.5-flash
🧠 Personality Traits:
 Here are the top 3 personality or cognitive traits:

1.  **Innovative:** This individual thrives on exploring 